# Maxwell's Curl Equations

Maxwell's curl equations are two different laws from the system of equations that model the functions of electromagnetism. They consist of the differential forms of Faraday's and Ampere's Laws, and are shown below.
$$
\begin{aligned}
  \nabla \times \vec{E} = -\frac{ \partial \vec{B} }{ \partial t} \\
  \nabla \times \vec{B} = \mu_0j + \frac{1}{c^2} \frac{\partial\vec{E}}{\partial t}
\end{aligned}
$$
These equations above are the ones commonly taught in physics classes, and are the differential forms of what we have seen in PY 202. But, for this problem working with H, or magnetic field density makes the equations simplify a little bit, and makes them easier to discretize. These rewritten equations look like:
$$
\begin{aligned}
  \nabla \times \vec{E} = -\mu_0\frac{ \partial \vec{H} }{ \partial t} \\
  \nabla \times \vec{H} = J + \epsilon_0\frac{\partial\vec{E}}{\partial t}
\end{aligned}
$$
In this situation, we are working with the functions of the magnetic and electric fields in one dimension, particularly where E is only in the x direction and H is only in the z direction, making the poynting vector of propogation in the y direction. First, we need to discretize curl. Curl is pretty messy in 3D, being
$$
\begin{aligned}
\nabla \times \vec{F} = \left( \begin{array}{ccc} \hat{i} & \hat{j} & \hat{k}\\ \frac{\partial}{\partial x} & \frac{\partial}{\partial y} & \frac{\partial}{\partial z} \\ P & Q & R \end{array}\right) \
\end{aligned}
$$
Which is equal to 
$$
\begin{aligned}
\left( \frac{\partial R}{\partial y} - \frac{\partial Q}{\partial z} \right) \hat{i} + \left( \frac{\partial P}{\partial z} - \frac{\partial R}{\partial x} \right) \hat{j} + \left( \frac{\partial Q}{\partial x} - \frac{\partial P}{\partial y} \right) \hat{k}
\end{aligned}
$$
What we can do though, is eliminate motion in two of the three directions, choosing H and E to oscillate in only one plane. All this does is change the sign in front of one piece of the equation, which depends varying on the direction you pick each to oscillate in, below is an example where H oscillates in the z direction and E oscillates in the x, but all other possible combinations of x,y and z would take the same form with a different sign. Here is the discretization.

$$
\begin{aligned}
  H(j,n) = H(j,n-1) + \frac{\Delta t}{\mu_0 \Delta y}(E(j+1,n) - E(j,n) \\
  E(j,n+1) = E(j,n) + \frac{\Delta t}{\epsilon \Delta y} (H(j,n) - H(j-1,n)
\end{aligned}
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Edit these for different results

jmax = 500
jsource = 250
nmax = 2500
lambda_0 = 550e-9
tau = 30
#====================================#
eps0 = 8.85e-12
mu0 = 1.26e-6
c = 1/np.sqrt(eps0*mu0)
Z = np.sqrt(mu0/eps0)
lambda_min = 350e-9
dx = lambda_min/20
dt = dx/c
t0 = tau*3
omega = 2*np.pi*c/lambda_0

In [ ]:
#Robbie
#input in the form (kx-wt) like the solution to the wave equation
def inp(freq,t,x):
  omega = 2*np.pi*freq
  k=omega/(c)
  return(k*x-omega*t)

#sin waveform
#general form
def gensin(amp,freq,t,num):
  return(amp*np.sin(2*np.pi*freq*t))

#in form of solution to the wave equation
def sin(amp,freq,t,x,num):
  return(gensin(amp,freq,inp(freq,t,x),num))


#random waveform
#general form
def genrand(amp,t,num):
  func = 0 #function is empty
  for i in range(num):
    ramp = np.random.rand()*np.random.rand()*5 #random amplitude
    romega = np.random.rand()*2*np.pi #random frequency
    func += ramp*np.sin(romega*t) #function is the composition of sines
  return(func)

#in form of solution to the wave equation
def rand(amp,freq,t,x,num):
  return(genrand(amp,inp(freq,t,x),num))


#square waveform
#general form
def gensquare(amp,freq,t,num):
  func = 0 #function is blank initially
  nmax = 2*num-1 #square wave Fourier Series only uses odd numbers
  index = np.arange(1,nmax,2) #index of odd numbers
  L = c/(2*freq) #half of the wavelength
  for n in index:
    func += (1/n)*np.sin(n*np.pi*t/L)
  return(amp*(4/np.pi)*func)

#in form of solution to the wave equation
def square(amp,freq,t,x,num):
  return(gensquare(amp,freq,inp(freq,t,x),num))


#triangle waveform
#general form
def gentriangle(amp,freq,t,num):
  func = 0 #function is blank initially
  nmax = 2*num-1 #triangle wave Fourier Series only uses odd numbers
  index = np.arange(1,nmax,2) #index of odd numbers
  L = c/(2*freq) #half of the wavelength
  for n in index:
    func += (((-1)**((n-1)/2))/n**2)*np.sin(n*np.pi*t/L)
  return(amp*(8/(np.pi)**2)*func)

#in form of solution to the wave equation
def triangle(amp,freq,t,x,num):
  return(gentriangle(amp,freq,inp(freq,t,x),num))


#sawtooth waveform
#general form
def gensawtooth(amp,freq,t,num):
  func = 0 #function is initially blank
  nmax = num #sawtooth waveform uses all integers
  index = np.arange(1,nmax,1) #index of integers
  L = c/(2*freq) #half of the wavelength
  for n in index:
    func += (1/n)*np.sin(n*np.pi*t/L)
  return(2*amp*(-1/(np.pi))*func)

#in form of solution to the wave equation
def sawtooth(amp,freq,t,x,num):
  return(gensawtooth(amp,freq,inp(freq,t,x),num))

In [ ]:
#Blake
#Input for the source in solutions of Maxwell's Equations
#Pick and un-comment the input of choice

def source(t):
  #return sin(t)
  #return triangle(1e-2,omega,t,0,500)
  #return square(1e9,omega,t,0,500)
  #return sawtooth(1e9,omega,t,0,500)
  #return rand(1,omega,t,0,500)
  return np.exp(-(t-t0)**2/tau**2)*np.sin(omega*t*dt)

In [ ]:
#FDTD Method
Ex = np.zeros([jmax+1,2])  
Hz = np.zeros([jmax+1,2])  
Eish = np.zeros([jmax+1,2])  
Hish = np.zeros([jmax+1,2])

for n in range(nmax):
  #Magnetic field boundaries
  Hz[jmax-1,1] = Hz[jmax-2,0]
  #Update Magnetic field
  for j in range(jmax-1):
    Hz[j,1] = Hz[j,0] + dt/(dx*mu0) * (Ex[j+1,1] - Ex[j,1])
    Hz[j,0] = Hz[j,1]
  #Magnetic field source
  Hz[jsource-1,1] -= source(n)/Z

  #Update Electric Field boundaries
  Ex[0,1] = Ex[1,0]
  #Update electric field
  for j in range(1,jmax):
    Ex[j,1] = Ex[j,0] + dt/(dx*eps0) * (Hz[j,1]-Hz[j-1,1])
    Ex[j,0] = Ex[j,1]

  #Electric field source
  Ex[jsource,1] += source(n+1)
  Ex[jsource,0] = Ex[jsource,1]

  if n%10 == 0:
    plt.plot(Ex[:,1])z
    plt.ylim([-1,1])
    plt.show()
    plt.close()

  #if n%50 == 0:
    #plt.plot(Hz[:,1])
    #plt.ylim([-0.002,0.002])
    #plt.show()
    #plt.close()

Below is an application of the idea used in the module on PDE's, doing a 2-step calculation. The equations modified, look like:

$$
\begin{aligned}
  \tilde H_z(j,n) = H_z(j,n-1) + \frac{\Delta t}{2\mu_0 \Delta y}(E_x(j+1,n) - E_x(j,n) \\
  \tilde E_x(j,n+1) = E_x(j,n) + \frac{\Delta t}{2\epsilon \Delta y} (H_z(j,n) - H_z(j-1,n)
\end{aligned}

\begin{aligned}
  H_z(j,n) = \tilde H_z(j,n-1) + \frac{\Delta t}{\mu_0 \Delta y}(\tilde E_x(j+1,n) - \tilde E_x(j,n) \\
  E_x(j,n+1) = \tilde E_x(j,n) + \frac{\Delta t}{\epsilon \Delta y} (\tilde H_z(j,n) - \tilde H_z(j-1,n)
\end{aligned}
$$
This equation could use some work, but it seems to work well for reflections and similar applications, the boundaries are finnicky at best, and make for a difficult time trying to recreate some of the infinitely propogating waves.

In [ ]:
jmax = 100
jsource = 50
nmax = 1000

#ish iteration loop
Ex = np.zeros([jmax+1,2])  
Hz = np.zeros([jmax+1,2])  
Eish = np.zeros([jmax+1,2])  
Hish = np.zeros([jmax+1,2])

for n in range(nmax):
  #Update Magnetic field-ish
  for j in range(jmax-1):
    Hish[j,1] = Hish[j,0] + dt/(2*dx*mu0) * (Eish[j+1,1] - Eish[j,1])
    Hish[j,0] = Hish[j,1]
    Hz[j,1] = Hish[j,0] + dt/(dx*mu0) * (Eish[j+1,1] - Eish[j,1])
    Hz[j,0] = Hish[j,1]
  #Magnetic field ish source
  Hish[jsource-1,1] -= source(n)/Z
  Hz[jsource-1,1] -= source(n)/Z

  #Update electric field-ish
  for j in range(1,jmax):
    Eish[j,1] = Eish[j,0] + dt/(2*dx*eps0) * (Hish[j,1]-Hish[j-1,1])
    Eish[j,0] = Eish[j,1]
    Ex[j,1] = Eish[j,0] + dt/(dx*eps0) * (Hish[j,1]-Hish[j-1,1])
    Ex[j,0] = Ex[j,1]

  #Electric field source
  Ex[jsource,1] += source(n+1)
  Ex[jsource,0] = Ex[jsource,1]

  if n%10 == 0:
    plt.plot(Ex[:,1])
    plt.ylim([-1,1])
    plt.show()
    plt.close()
  
  #if n%50 == 0:
    #plt.plot(Hz[:,1])
    #plt.ylim([-0.002,0.002])
    #plt.show()
    #plt.close()

The block below is an application of the first block of code, but this time with no boundaries allowing perfect reflection off the sides of the graph, to compare and contrast the above block with this one. What we can see is that there is a lot less precision in the below block, with plenty of sharp lines and peaks.

In [ ]:
#FDTD Method (bounce)
Ex = np.zeros([jmax+1,2])  
Hz = np.zeros([jmax+1,2])  
Eish = np.zeros([jmax+1,2])  
Hish = np.zeros([jmax+1,2])
for n in range(nmax):
  #Magnetic field boundaries
  #Hz[jmax-1,1] = Hz[jmax-2,0]
  #Update Magnetic field
  for j in range(jmax-1):
    Hz[j,1] = Hz[j,0] + dt/(dx*mu0) * (Ex[j+1,1] - Ex[j,1])
    Hz[j,0] = Hz[j,1]
  #Magnetic field source
  Hz[jsource-1,1] -= source(n)/Z

  #Update Electric Field boundaries
  #Ex[0,1] = Ex[1,0]
  #Update electric field
  for j in range(1,jmax):
    Ex[j,1] = Ex[j,0] + dt/(dx*eps0) * (Hz[j,1]-Hz[j-1,1])
    Ex[j,0] = Ex[j,1]

  #Electric field source
  Ex[jsource,1] += source(n+1)
  Ex[jsource,0] = Ex[jsource,1]

  # if n%10 == 0:
  #   plt.plot(Ex[:,1])
  #   plt.ylim([-1,1])
  #   plt.show()
  #   plt.close()

  #if n%10 == 0:
    #plt.plot(Ex)
    #plt.ylim([-1,1])
    #plt.show()
    #plt.close()

The block below is a modification of the above algorithm to be similar to what we worked in the module with the boundary value problem, where this version of the code has been edited to only work in 1 loop, rather than 3, utilizing indexing in the solution.

In [ ]:
#Speediest Code
Ex = np.zeros([jmax+1,2])  
Hz = np.zeros([jmax+1,2])  

for n in range(nmax):
  #Magnetic field boundaries
  Hz[jmax-1,1] = Hz[jmax-2,0]
  #Update Magnetic field
  Hz[:jmax-1,1] = Hz[:jmax-1,0] + dt/(dx*mu0) * (Ex[1:jmax,1] - Ex[:jmax-1,1])
  Hz[:,0] = Hz[:,1]
  #Magnetic field source
  Hz[jsource-1,1] -= source(n)/Z

  #Update Electric Field boundaries
  Ex[0,1] = Ex[1,0]
  #Update electric field
  Ex[1:jmax,1] = Ex[1:jmax,0] + dt/(dx*eps0) * (Hz[1:jmax,1]-Hz[:jmax-1,1])
  Ex[:,0] = Ex[:,1]

  #Electric field source
  Ex[jsource,1] += source(n+1)
  Ex[jsource,0] = Ex[jsource,1]

  # if n%10 == 0:
  #   plt.plot(Ex[:,1])
  #   plt.ylim([-1,1])
  #   plt.show()
  #   plt.close()

We know two things about the relationships between the two aspects of the problem here.

$$
\begin{aligned}
  B = \mu H \\
  E = cB
\end{aligned}
$$

From this we can discern that:

$$
\begin{aligned}
  E = c\mu H
\end{aligned}
$$

We can use this axiom to determine whether our E and H values are proportioned correctly.

In [ ]:
#Work Checking
Ex = np.zeros([jmax+1,2])  
Hz = np.zeros([jmax+1,2])  
for n in range(nmax):
  #Magnetic field boundaries
  Hz[jmax-1,1] = Hz[jmax-2,0]
  #Update Magnetic field
  for j in range(jmax-1):
    Hz[j,1] = Hz[j,0] + dt/(dx*mu0) * (Ex[j+1,1] - Ex[j,1])
    Hz[j,0] = Hz[j,1]
  #Magnetic field source
  Hz[jsource-1,1] -= source(n)/Z

  #Update Electric Field boundaries
  Ex[0,1] = Ex[1,0]
  #Update electric field
  for j in range(1,jmax):
    Ex[j,1] = Ex[j,0] + dt/(dx*eps0) * (Hz[j,1]-Hz[j-1,1])
    Ex[j,0] = Ex[j,1]

  #Electric field source
  Ex[jsource,1] += source(n+1)
  Ex[jsource,0] = Ex[jsource,1]

error = Ex[:,:]/(c*mu0) - Hz[:,:]

if error.all() <= 1e-5:
  print(True)
print(error)

We are able to acheive a relatively small amount of error from the piece of code above, on average being around 1x10^-19.

In [ ]:
Ex = np.zeros([jmax+1,2])  
Hz = np.zeros([jmax+1,2])  
Eish = np.zeros([jmax+1,2])  
Hish = np.zeros([jmax+1,2])
#full iteration
for n in range(nmax):
  #Update Magnetic field-ish and magnetic field
  for j in range(jmax-1):
    Hish[j,1] = Hish[j,0] + dt/(2*dx*mu0) * (Eish[j+1,1] - Eish[j,1])
    Hish[j,0] = Hish[j,1]
    Hz[j,1] = Hish[j,0] + dt/(dx*mu0) * (Eish[j+1,1] - Eish[j,1])
    Hz[j,0] = Hish[j,1]
  #Magnetic field ish source
  Hish[jsource-1,1] -= source(n)/Z
  Hz[jsource-1,1] -= source(n)/Z

  #Update electric field-ish and electric field
  for j in range(1,jmax):
    Eish[j,1] = Eish[j,0] + dt/(2*dx*eps0) * (Hish[j,1]-Hish[j-1,1])
    Eish[j,0] = Eish[j,1]
    Ex[j,1] = Eish[j,0] + dt/(dx*eps0) * (Hish[j,1]-Hish[j-1,1])
    Ex[j,0] = Eish[j,1]

  #Electric field source
  Ex[jsource,1] += source(n+1)
  Ex[jsource,0] = Ex[jsource,1]

  #if n%5 == 0:
    #plt.plot(Ex[:,1])
    #plt.ylim([-1,1])
    #plt.show()
    #plt.close()

error = Ex[:,:]/(c*mu0) - Hz[:,:]

print(error)

What is interesting here is that although this code makes the smoother, more visually pleasing graph, it is actually less accurate on average.

# References

https://en.wikipedia.org/wiki/Finite-difference_time-domain_method \
https://www.synopsys.com/glossary/what-is-fdtd.html#:~:text=The%20Finite%2DDifference%20Time%2DDomain,of%20the%20computing%20power%20available. \
https://speag.swiss/products/semcad/modules/what-is-fdtd/ \
https://my.ece.utah.edu/~ece6340/LECTURES/lecture%2014/FDTD.pdf
https://www.youtube.com/watch?v=S-6Z8N-30AU \
These final two sources have the same discretization, which is the one I ended up basing my code around.